In [ ]:
import sys
relative_root = "../.."
sys.path.append(relative_root)

In [ ]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score
import json
from examples.training import titanic_config
from pathlib import Path

## Load data & model

In [ ]:
try:
    preprocessed_train = pd.read_csv(Path(relative_root) / titanic_config.LOCAL_TRAIN_FILENAME)
    preprocessed_test = pd.read_csv(Path(relative_root) / titanic_config.LOCAL_TEST_FILENAME)
    rf_model = joblib.load(Path(relative_root) / titanic_config.LOCAL_MODEL_FILENAME)
    with open(Path(relative_root) / titanic_config.LOCAL_FI_FILENAME, "r") as file:
        feature_importance = json.loads(file.read())
except FileNotFoundError:
    raise FileNotFoundError("To generate these files, run `make train-titanic`")

In [ ]:
rf_model.predict(preprocessed_test)[0]  # test model inference

## Init trubrics context

In [ ]:
from trubrics.context import DataContext, ModelContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version=0.1,
    training_data=preprocessed_train,
    testing_data=preprocessed_test,
    target_column=titanic_config.TARGET
)
model_context = ModelContext(
    name="my_first_model",
    version=0.1,
    estimator=rf_model,
    evaluation_function=accuracy_score
)

## Init trubrics validator

In [ ]:
from trubrics.validators.base import Validator

In [ ]:
model_validator = Validator(data=data_context, model=model_context)

## Use the trubrics validator to create out-of-the-box validations

In [ ]:
edge_case = {
    "Sex": "male",
    "Embarked": "S",
    "Title": "Master",
    "Pclass": 2,
    "Age": 28,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 37
}

In [ ]:
robustness = [
    model_validator.validate_single_edge_case(edge_case_data=edge_case, desired_output=0), # example of fail
    model_validator.validate_single_edge_case(edge_case_data=edge_case, desired_output=1) # example of pass
]
robustness[0].severity = "warning"  # change the severity of a validation
robustness[1].severity = "experiment"
robustness[0].dict()

In [ ]:
performance = [
    model_validator.validate_performance_against_threshold(threshold=0.8),
    model_validator.validate_performance_against_threshold(threshold=0.75)
]
performance[0].dict()

In [ ]:
fairness = [
    model_validator.validate_biased_performance_across_category(category="Embarked", threshold=0.1),
    model_validator.validate_biased_performance_across_category(category="Sex", threshold=0.05)
]
fairness[0].dict()

In [ ]:
explainability = [
    model_validator.validate_feature_in_top_n_important_features(feature="Sex_female", feature_importance=feature_importance, top_n_features=3),
    model_validator.validate_feature_in_top_n_important_features(feature="Age", feature_importance=feature_importance, top_n_features=2)
]
explainability[0].dict()

## Collect user feedback from app

In [ ]:
from trubrics.utils.loader import get_business_feedback_data
# read test data and run single outlier test
try:
    data = get_business_feedback_data(tracking=False)
    display(data)
except FileNotFoundError:
    print("Please generate feedback from the streamlit app in order to read it back here")

--> **DS response: "It isn't normal, the model should not be more accurate for different groups of people. I'll add a test for this."**

## Create custom validations

In [ ]:
from trubrics.validators.validation_output import validation_output, validation_output_type


class CustomValidator(Validator):
    def __init__(self, data: DataContext, model: ModelContext):
        super().__init__(data, model)
        
    @validation_output
    def validate_performance_for_different_fares(self, fare_cutoff):
        return self._validate_performance_for_different_fares(fare_cutoff)

    def _validate_performance_for_different_fares(self, fare_cutoff: int = 50) -> validation_output_type:
        """
        Write your custom validation function here.
        
        Notes
        -----
            This method is separated from validate_performance_for_different_fares
            to apply @validation_output and for unit testing.

            The @validation_output decorator allows you to generate a Validation object,
            and must be used to be able to save your validation as part of a Trubric.
            This decorator requires you to return values with the same type as validation_output_type.
        """
        
        errors_df = self.trubrics_model.explore_test_set_errors()
        number_of_errors_by_fare_ratio = (
            errors_df.loc[lambda x: x["Fare"] <= fare_cutoff].shape[0]
            / errors_df.loc[lambda x: x["Fare"] > fare_cutoff].shape[0]
        )
        return (
            number_of_errors_by_fare_ratio > 0.5 and number_of_errors_by_fare_ratio < 1.5,
            {"number_of_errors_by_fare_ratio": round(number_of_errors_by_fare_ratio, 3)}
        )

In [ ]:
model_custom_validator = CustomValidator(data=data_context, model=model_context)

In [ ]:
custom = [model_custom_validator.validate_performance_for_different_fares(fare_cutoff=25)]
custom[0].severity = "warning"  # change the severity of a validation
custom[0].dict()

## Save trubric

In [ ]:
from trubrics.context import TrubricContext

validations = robustness + performance + fairness + explainability + custom

trubric_context = TrubricContext(
    name="my_first_trubric",
    model_context_name=model_context.name,
    model_context_version=model_context.version,
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    validations=validations,
)

trubric_context.save_local(path="../data")

In [ ]:
# or save to trubrics UI
try:
    trubric_context.save_ui(local_port=8000)
except:
    print("Looks like the Trubrics API is not currently deployed.")

In [ ]:
trubric_context.dict()

## Execute trubric from file

In [ ]:
from trubrics.validators.run import run_trubric

In [ ]:
all_validation_results = run_trubric(
    data_context=data_context,
    model_context=model_context,
    custom_validator=model_custom_validator,
    trubric_path="../data/my_first_trubric.json"
)

In [ ]:
for validation_result in all_validation_results:
    print(validation_result)